In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## importing all the required libraries 

In [11]:
from infrastructure import *
from converting_file_zip import *
import boto3
import os
import json

## clients

loading all the required clients from the boto3

In [12]:
s3_client = boto3.client('s3') #s3 client
glue_client = boto3.client('glue') #glue client
iam_client = boto3.client('iam') #IAM client
lambda_client = boto3.client('lambda') #lambda client

## S3 bucket

creating the s3 resource to store raw data

In [13]:
data_bucket_name = 'pn-youtube-raw-bucket' # raw data s3 bucket name

In [14]:
createBucket(s3_client, data_bucket_name) # creating the raw s3 bucket

The bucket pn-youtube-raw-bucket is created


## Uploading data to s3 bucket

uploading csv files

In [15]:
try:
    s3_client.upload_file('./sat_data/CAvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=ca/CAvideos.csv') # uploading file to raw data s3 bucket
    s3_client.upload_file('./sat_data/DEvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=de/DEvideos.csv')
    s3_client.upload_file('./sat_data/FRvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=fr/FRvideos.csv')
    s3_client.upload_file('./sat_data/GBvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=gb/GBvideos.csv')
    s3_client.upload_file('./sat_data/INvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=in/INvideos.csv')
    s3_client.upload_file('./sat_data/JPvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=jp/JPvideos.csv')
    s3_client.upload_file('./sat_data/KRvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=kr/KRvideos.csv')
    s3_client.upload_file('./sat_data/MXvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=mx/MXvideos.csv')
    s3_client.upload_file('./sat_data/RUvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=ru/RUvideos.csv')
    s3_client.upload_file('./sat_data/USvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=us/USvideos.csv')
    s3_client.upload_file('./sat_data/DEvideos.csv', data_bucket_name, 'youtube/raw_statistics/region=de/DEvideos.csv')
    print('csv files are successfully upload to s3 storage')
except Exception as e:
    print(f'error occured while uloading the csv files: {e}')

csv files are successfully upload to s3 storage


uploading json files 

In [29]:
file_lst = os.listdir('./sat_data/') # listing the all the file in the folder 
json_file_lst =[]
for file_name in file_lst:
    if file_name.endswith('.json'): # listing the all the json files in the folder
        json_file_lst.append(file_name)
    else:
        pass

In [58]:
try:
    for file_name in json_file_lst:
        s3_client.upload_file(f'./sat_data/{file_name}', data_bucket_name, f'youtube/raw_statistics_reference_data/{file_name}') # uploading file to raw data s3 bucket
        print('json filess are successfully upload to s3 storage')
except Exception as e:
    print(f'error occured while uloading the csv files: {e}')

json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage
json filess are successfully upload to s3 storage


# glue Crawler 

creating a glue crawler

resources need to create a glue crawler<br>
role: glue crawler resource need permission to s3 bucket<br> 
glue database: to store glue catalog <br>



Creating a role to give glue access the s3 bucket 

In [14]:
glue_s3_role_name = 'pn-youtube-glue-s3-role' # glue to s3 role name 
glue_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "glue.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}                                                                   ## policy attachment

s3_arn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'  ## s3 policy attachment 
glue_service_arn = 'arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole' # glue service plolicy attachment
raw_glue_db = 'pn-youtube-raw-db' # raw database name

In [13]:
createIamRole(iam_client, glue_s3_role_name, glue_policy_document) # creating a role
attachRolePolicies(iam_client, glue_s3_role_name, s3_arn) # attaching s3 policy
attachRolePolicies(iam_client, glue_s3_role_name, glue_service_arn) #attaching glue service policy
createGlueDatabase(glue_client, raw_glue_db) # creating a raw database

NameError: name 'glue_s3_role_name' is not defined

Creating a json Glue Crawler

In [71]:
s3_json_data_path = 's3://pn-youtube-raw-bucket/youtube/raw_statistics_reference_data/'
glue_s3_crawler_name = 'pn-youtube-raw-crawler'

In [13]:
createGlueCrawler(glue_client, glue_s3_crawler_name, glue_s3_role_name, raw_glue_db, s3_json_data_path)

pn-youtube-raw-crawler is succussfully created


Execute the json glue crawler 

In [14]:
startCrawler(glue_client, glue_s3_crawler_name)

pn-youtube-raw-crawler is successfully started


Check the status of json glue crawler

In [29]:
statusCrawler(glue_client, glue_s3_crawler_name)

The glue crawler status :SUCCEEDED
The glue crawler catalog are store in :pn-youtube-raw-crawler


## athena 

athena to view and query the data catalog

Before crawling the data we need to make some modification to json file and store it as a parquet file

## Lambda function

create a function to convert json file to parquet file

resources required for lambda funnction

role: lambda require access to s3 and glue

s3 bucket : to store parquet files

glue database : to store cleansed catalog data 


In [15]:
lambda_s3_role_name = 'pn-youtube-lambda-s3-role'
lambda_policy_document ={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sts:AssumeRole"
            ],
            "Principal": {
                "Service": [
                    "lambda.amazonaws.com"
                ]
            }
        }
    ]
}

s3_arn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'  ## s3 policy attachment

glue_service_arn = 'arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole' # glue policy attachment

cleansed_bucket_name = 'pn-youtube-cleansed-data' # cleansed s3 bucket name
cleansed_database_name = 'pn-youtube-cleansed-db' #cleansed glue daatabase name

In [16]:
createIamRole(iam_client,lambda_s3_role_name,lambda_policy_document) # creating  a role for lambda 
attachRolePolicies(iam_client,lambda_s3_role_name, s3_arn) # attaching s3 policy to lambda role
attachRolePolicies(iam_client,lambda_s3_role_name, glue_service_arn) # attaching the glue policy to lambda role
createBucket(s3_client, cleansed_bucket_name) #creating a cleansed s3 bucket
createGlueDatabase(glue_client, cleansed_database_name) #creating a cleansed glue database

role pn-youtube-lambda-s3-role is created successfully
policy is successfully attached to pn-youtube-lambda-s3-role
policy is successfully attached to pn-youtube-lambda-s3-role
The bucket pn-youtube-cleansed-data is created
database pn-youtube-cleansed-db is created successfully


Code to upload to lambda function<br>
'convert_json_parquet.py' has the code for converting json to parquet and storing it in s3 bucket and creating a glue catalog

In [52]:
py_file = 'convert_json_parquet.py'
zip_file = 'convert_json_parquet.zip'
convertFiletoZipfile(py_file, zip_file)

file convert_json_parquet.py is successfully converted into zipfile convert_json_parquet.zip


In [53]:
with open(r'./convert_json_parquet.zip', 'rb') as file:
    file_byte = file.read()
json_parquet_func_name = 'pn-youtube-json-parquet-lambda'
kernal_name = 'python3.12'
lambda_s3_role_arn = 'arn:aws:iam::482268324100:role/pn-youtube-lambda-s3-role'
json_parquest_handler = 'convert_json_parquet.lambda_handler'

Creating and updating the lambda function

In [54]:
createLambdaFunction(lambda_client, json_parquet_func_name, kernal_name , lambda_s3_role_arn, json_parquest_handler, file_byte)

lambda function pn-youtube-json-parquet-lambda is created successfully


update the configuration details 

running time : 3 min or 180 seconds <br>
memory : 512<br>
layer :arn:aws:lambda:eu-north-1:336392948345:layer:AWSSDKPandas-Python312:19

In [55]:
lambda_client.update_function_configuration(
    FunctionName = json_parquet_func_name,
    Timeout = 180,
    MemorySize = 512,
    Layers=[
        'arn:aws:lambda:eu-north-1:336392948345:layer:AWSSDKPandas-Python312:19'
    ],
    Environment = {
        'Variables':{
            'glue_catalog_db_name' : os.environ.get('glue_catlog_db_name'),
            'glue_catalog_table_name' : os.environ.get('glue_catalog_table_name'),
            's3_cleansed_layer' : os.environ.get('s3_cleansed_layer'),
            'write_data_operation': os.environ.get('write_data_operation')
        }    
    }    
)

{'ResponseMetadata': {'RequestId': '2ac99838-9b5d-4426-9d35-98fd2809cd6a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 14 Oct 2025 11:46:11 GMT',
   'content-type': 'application/json',
   'content-length': '1813',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2ac99838-9b5d-4426-9d35-98fd2809cd6a'},
  'RetryAttempts': 0},
 'FunctionName': 'pn-youtube-json-parquet-lambda',
 'FunctionArn': 'arn:aws:lambda:eu-north-1:482268324100:function:pn-youtube-json-parquet-lambda',
 'Runtime': 'python3.12',
 'Role': 'arn:aws:iam::482268324100:role/pn-youtube-lambda-s3-role',
 'Handler': 'convert_json_parquet.lambda_handler',
 'CodeSize': 1663,
 'Description': '',
 'Timeout': 180,
 'MemorySize': 512,
 'LastModified': '2025-10-14T11:46:10.000+0000',
 'CodeSha256': 'Y9fj0dwv1i1BxApXwpNXuYxG4cTGu7Sf7bMT0J3dur8=',
 'Version': '$LATEST',
 'Environment': {'Variables': {'glue_catalog_table_name': 'cleaned-statistics-reference-data',
   's3_cleansed_layer': 's3://pn-youtube-cleansed-data

Running the lambda function

In [ ]:
for json_file in json_file_lst:
    payload = {
  "Records": [
    {
      "eventVersion": "2.0",
      "eventSource": "aws:s3",
      "awsRegion": "eu-north-1",
      "eventTime": "1970-01-01T00:00:00.000Z",
      "eventName": "ObjectCreated:Put",
      "userIdentity": {
        "principalId": "EXAMPLE"
      },
      "requestParameters": {
        "sourceIPAddress": "127.0.0.1"
      },
      "responseElements": {
        "x-amz-request-id": "EXAMPLE123456789",
        "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH"
      },
      "s3": {
        "s3SchemaVersion": "1.0",
        "configurationId": "testConfigRule",
        "bucket": {
          "name": 'pn-youtube-raw-bucket',
          "ownerIdentity": {
            "principalId": "EXAMPLE"
          },
          "arn": "arn:aws:s3:::pn-youtube-raw-bucket"
        },
        "object": {
          "key": f"youtube/raw_statistics_reference_data/{json_file}",
          "size": 1024,
          "eTag": "0123456789abcdef0123456789abcdef",
          "sequencer": "0A1B2C3D4E5F678901"
        }
      }
    }
  ]
}
    lambdaFuncInvoke(lambda_client, json_parquet_func_name, payload)

there is no direct trigger API for lambda in boto3 <br>
We can configure add permissions to s3 and put notification congiguration in s3 for triggering the lambda function<br>
we can also add directly the trigger in AWS Console

In [37]:
statement_id = 's3'
source_arn ='arn:aws:s3:::pn-youtube-raw-bucket'
notification_configurations = {
    'LambdaFunctionConfigurations':[
        {
        'LambdaFunctionArn' : 'arn:aws:lambda:eu-north-1:482268324100:function:pn-youtube-json-parquet-lambda',
        'Events':[
            's3:ObjectCreated:*'
        ],
        'Filter':{
            'Key':{
                'FilterRules':[
                    {
                        'Name':'Suffix',
                        'Value':'.json'
                    }
                ]
            }
        }
    }
    ]
}
principal_name ='s3.amazonaws.com'

In [ ]:
addPermission(lambda_client, json_parquet_func_name, statement_id, principal_name, source_arn)
putBucketNotificationConfiguration(s3_client, data_bucket_name, notification_configurations)

Permission to s3 for pn-youtube-json-parquet-lambda is added successfully


Converting csv file to glue catalog

In [69]:
glue_s3_csv_crawler_name = 'pn-youtube-csv-crawler'
s3_csv_data_path = 's3://pn-youtube-raw-bucket/youtube/raw_statistics/'

Creating a csv glue crawler

In [73]:
createGlueCrawler(glue_client, glue_s3_csv_crawler_name, glue_s3_role_name, raw_glue_db, s3_csv_data_path)

pn-youtube-csv-crawler is succussfully created


Running the csv glue crawler 

In [74]:
startCrawler(glue_client,glue_s3_csv_crawler_name)

pn-youtube-csv-crawler is successfully started


Checking the status of csv glue crawler 

In [30]:
statusCrawler(glue_client, glue_s3_csv_crawler_name)

The glue crawler status :SUCCEEDED
The glue crawler catalog are store in :pn-youtube-csv-crawler


athena <br>

query with athena to check the data 

create a bucket to store scripts

In [10]:
script_bucket_name = 'pn-youtube-code-scripts-bucket'

In [61]:
createBucket(s3_client, script_bucket_name)

The bucket pn-youtube-code-scripts-bucket is created


In [11]:
try:
    s3_client.upload_file('./convert_csv_parquet.py', script_bucket_name, "csv/convert_csv_parquet.py")
    print("CSV script file is successfully uploaded")
except Exception as e:
    print(f"The following error occurred: {e}")

CSV script file is successfully uploaded


In [ ]:
try:
    s3_client.upload_file('./convert_json_parquet.py', script_bucket_name, "json/convert_json_parquet.py")
    print("Json script file is successfully uploaded")
except Exception as e:
    print(f"The following error occurred: {e}")

CSV script file is successfully uploaded


creating JOB in boto3 is complex when compared to do it in Console

In [7]:
job_name = 'youtube-csv-parquet-ETL-job'
role_assigned = 'arn:aws:iam::482268324100:role/pn-youtube-glue-s3-role'
script_name = 'glueetl'
script_location='s3://pn-youtube-code-scripts-bucket/csv/convert_csv_parquet.py'

In [12]:
createJob(glue_client, job_name, role_assigned, script_name, script_location)

In [ ]:
csv_cleansed_crawler_name = 'pn-youtube-cleansed-csv-crawler'
cleansed_csv_parquet_path = 's3://pn-youtube-cleansed-data/youtube/raw_statistics/'

In [17]:
createGlueCrawler(glue_client, csv_cleansed_crawler_name, glue_s3_role_name, cleansed_database_name, cleansed_bucket_name)

pn-youtube-cleansed-csv-crawler is succussfully created


In [18]:
startCrawler(glue_client, csv_cleansed_crawler_name)

pn-youtube-cleansed-csv-crawler is successfully started
